If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8019/tree?token=project-segmented-images```  

This notebook projects segmented panoramas onto an input mesh.describes a pipeline to download panoramas and their corresponding depth maps from Google Street View.  
It takes an input mesh file and segmented panoramas as ```.json```s with ```List[List[int]]```, and returns an output segmented point cloud as a ```.csv```.  
It also takes an input ```.csv``` describing the latitude and longitude of the input point clouds.  
The mesh is assumed to be geolocated by EPSG:3414.  

In [102]:
target_dir = "data"
panorama_dir = "thomson-segmented"
output_dir = "thomson-segmented-projected"
mesh_path = "projection-mesh.obj"
coords_path = "thomson-id.csv"
# projection_length = 70

In [103]:
import meshlib
from pyproj import Transformer

import csv
import json
import math
import os
from pathlib import Path

In [104]:
def pixel_to_point(x, y, img_width, img_height, length,
                  x_range=(-1.0, 1.0), y_range=(-1.0, 1.0), heading=0.0):
    pi = math.pi
    sin = math.sin
    cos = math.cos
    
    x0 = x_range[0]
    dx = x_range[1] - x0
    y0 = y_range[0]
    dy = y_range[1] - y0
    
    x_norm = ((x + 0.5) / img_width) * dx + x0
    y_norm = ((y + 0.5) / img_height) * dy + y0
    theta = -1.0 * pi * x_norm
    phi = -1.0 * pi / 2 * y_norm
    point = [
        -1.0 * length * sin(theta - heading) * cos(phi),
        length * cos(theta - heading) * cos(phi),
        length * sin(phi)
    ]
    
    return point

In [ ]:
transformer = Transformer.from_crs(4326, 3414, always_xy=True)
coords = {}
with open(os.path.join(target_dir, coords_path), 'r') as fp:
    reader = csv.reader(fp)
    next(reader)
    for row in reader:
        lat = float(row[1])
        lon = float(row[2])
        xy = transformer.transform(lon, lat)
        coords[row[0]] = xy

mesh = meshlib.mrmeshpy.loadMesh(os.path.join(target_dir, mesh_path))

segmentation_paths = []
for root, dirs, files in os.walk(os.path.join(target_dir, panorama_dir)):
    segmentation_paths.extend(files)
    break

k = 0
Path(os.path.join(target_dir, output_dir)).mkdir(parents=True, exist_ok=True)
for segmentation_path in segmentation_paths:
    id = '.'.join(segmentation_path.split('.')[:-1])
    path = os.path.join(target_dir, panorama_dir, segmentation_path)

    if k % 100 == 0:
        print(k + 1, id)

    with open(path, 'r') as fp:
        segmentation = json.load(fp)
    img_height = len(segmentation)
    img_width = len(segmentation[0])
    points = []

    for i in range(img_height):
        for j in range(img_width):
            point = pixel_to_point(j, i, img_width, img_height, 1)
            origin = coords[id]
            p = meshlib.mrmeshpy.Vector3f(origin[0], origin[1], 0.0)
            d = meshlib.mrmeshpy.Vector3f(point[0], point[1], point[2])
            line = meshlib.mrmeshpy.Line3f(p, d)
            intersection = meshlib.mrmeshpy.rayMeshIntersect(mesh, line, rayEnd=70)
            if intersection:
                point = intersection.proj.point
                label = segmentation[i][j]
                points.append([point.x, point.y, point.z, label])
    
    with open(os.path.join(target_dir, output_dir, f"{id}.csv"), 'w') as fp:
        writer = csv.writer(fp)
        writer.writerows(points)

    k += 1

1 --Y9DvKG_QeCvB9fkR2L_Q
